In [0]:
employee_schema=["employee_id","name","doj","dept_id","gender","salary"]
employee_data=[(10,"Raj","1991","100","M",2000),
               (20,"Rahul","2002","200","M",8000),
               (30,"Raghav","2010","100","",6000),
               (40,"Raja","2004","100","F",7000),
               (50,"Rama","2008","400","F",1000)]
employdf=spark.createDataFrame(data=employee_data,schema=employee_schema)
display(employdf)


employee_id,name,doj,dept_id,gender,salary
10,Raj,1991,100,M,2000
20,Rahul,2002,200,M,8000
30,Raghav,2010,100,,6000
40,Raja,2004,100,F,7000
50,Rama,2008,400,F,1000


In [0]:
depart_data=[("HR",100),
             ("supply",200),
             ("sales",300),
             ("stock",400)]
depart_schema=["dept_name","dept_id"]
departdf=spark.createDataFrame(data=depart_data,schema=depart_schema)
display(departdf)

dept_name,dept_id
HR,100
supply,200
sales,300
stock,400


In [0]:
from pyspark.sql.functions import col
dfjoin=employdf.join(departdf,employdf.dept_id==departdf.dept_id,"inner")\
    .withColumn('bonus',col('salary')*0.01)\
        .groupBy("dept_name").sum('salary').alias("total")
display(dfjoin)

dept_name,sum(salary)
stock,1000
HR,15000
supply,8000


In [0]:
dfjoin.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   ResultQueryStage 3, Statistics(sizeInBytes=108.0 B, rowCount=3, ColumnStat: N/A, isRuntime=true)
   +- *(4) HashAggregate(keys=[dept_name#27], functions=[finalmerge_sum(merge sum#292L) AS sum(salary#7L)#285L])
      +- AQEShuffleRead coalesced
         +- ShuffleQueryStage 2, Statistics(sizeInBytes=96.0 B, rowCount=3, ColumnStat: N/A, isRuntime=true)
            +- Exchange hashpartitioning(dept_name#27, 200), ENSURE_REQUIREMENTS, [plan_id=560]
               +- *(3) HashAggregate(keys=[dept_name#27], functions=[partial_sum(salary#7L) AS sum#292L])
                  +- *(3) Project [salary#7L, dept_name#27]
                     +- *(3) SortMergeJoin [cast(dept_id#5 as bigint)], [dept_id#28L], Inner
                        :- Sort [cast(dept_id#5 as bigint) ASC NULLS FIRST], false, 0
                        :  +- AQEShuffleRead coalesced
                        :     +- ShuffleQueryStage 0, Statistics(sizeInBy

In [0]:
dfjoin.explain(extended=True)

== Parsed Logical Plan ==
SubqueryAlias total
+- Aggregate [dept_name#27], [dept_name#27, sum(salary#7L) AS sum(salary)#379L]
   +- Project [employee_id#2L, name#3, doj#4, dept_id#5, gender#6, salary#7L, dept_name#27, dept_id#28L, (cast(salary#7L as double) * 0.01) AS bonus#359]
      +- Join Inner, (cast(dept_id#5 as bigint) = dept_id#28L)
         :- LogicalRDD [employee_id#2L, name#3, doj#4, dept_id#5, gender#6, salary#7L], false
         +- LogicalRDD [dept_name#27, dept_id#28L], false

== Analyzed Logical Plan ==
dept_name: string, sum(salary): bigint
SubqueryAlias total
+- Aggregate [dept_name#27], [dept_name#27, sum(salary#7L) AS sum(salary)#379L]
   +- Project [employee_id#2L, name#3, doj#4, dept_id#5, gender#6, salary#7L, dept_name#27, dept_id#28L, (cast(salary#7L as double) * 0.01) AS bonus#359]
      +- Join Inner, (cast(dept_id#5 as bigint) = dept_id#28L)
         :- LogicalRDD [employee_id#2L, name#3, doj#4, dept_id#5, gender#6, salary#7L], false
         +- LogicalRDD [de

In [0]:
#dfjoin.explain(mode="simple")
#dfjoin.explain(mode="extended")
#dfjoin.explain(mode="formatted")
dfjoin.explain(mode="cost")

== Optimized Logical Plan ==
Aggregate [dept_name#27], [dept_name#27, sum(salary#7L) AS sum(salary)#379L], Statistics(sizeInBytes=8.18E+35 B, ColumnStat: N/A)
+- Project [salary#7L, dept_name#27], Statistics(sizeInBytes=8.18E+35 B, ColumnStat: N/A)
   +- Join Inner, (cast(dept_id#5 as bigint) = dept_id#28L), Statistics(sizeInBytes=1.45E+36 B, ColumnStat: N/A)
      :- Project [dept_id#5, salary#7L], Statistics(sizeInBytes=2.8 EiB, ColumnStat: N/A)
      :  +- Filter isnotnull(dept_id#5), Statistics(sizeInBytes=8.0 EiB, ColumnStat: N/A)
      :     +- LogicalRDD [employee_id#2L, name#3, doj#4, dept_id#5, gender#6, salary#7L], false, Statistics(sizeInBytes=8.0 EiB, ColumnStat: N/A)
      +- Filter isnotnull(dept_id#28L), Statistics(sizeInBytes=8.0 EiB, ColumnStat: N/A)
         +- LogicalRDD [dept_name#27, dept_id#28L], false, Statistics(sizeInBytes=8.0 EiB, ColumnStat: N/A)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   HashAggregate(keys=[dept_name#27